####File: ReadExpData.ipynb
- This file will read all video you provide, extract experiment data, and put it into corresponding excel
- The main function `readExpData()` is at the bottom. **Please ensure that all of code above `readExpData()` has been compiled and run before launching the main function.**
- Just compile and run sequentially.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as functional
import torchvision 
from torchvision.datasets import ImageFolder
import os
from xlwt import Workbook
from torch.utils.data import Dataset
from PIL import Image
import sys

In [ ]:
custom_transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5 ), (0.5))],
    )
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

In [ ]:
class Network(nn.Module):
    def __init__(self):
        self.output_size = 10
        
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2)
        self.fc = nn.Linear(400, 120)
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, self.output_size)
        
    def forward(self, x):
        x = self.pool(functional.relu(self.conv1(x)))
        x = self.pool(functional.relu(self.conv2(x)))
        x = x.view(-1, 400)
        x = functional.relu(self.fc(x))
        x = functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class MyDataset_fromdisk(Dataset):
    def __init__(self, dirpath, transform = None, target_transform = None):
        # dirpath='./frames/1'
        dirlist = os.listdir(dirpath)
        dirlist.sort(key=lambda x:(eval(x[:-6]), x[:-4]))
        # print(dirlist)
        # fh = open(txt_path, 'r')
        imgs = []
        for imgtag in dirlist:
            imgs.append((dirpath + imgtag, 1)) # all tag of images is 1 because we don't use tag
        self.imgs = imgs 
        self.transform = transform
        self.target_transform = target_transform
        
    def __getitem__(self, index):
        fn, label = self.imgs[index]
        # img = Image.open(fn) #no need to convert?
        img = Image.open(fn).convert('RGB') 
        if self.transform is not None:
            img = self.transform(img) 
        return img, label
    def __len__(self):
        return len(self.imgs)

class MyDataset_notfromdisk(Dataset):
    def __init__(self, imglist, transform = None, target_transform = None):
        imgs = []
        for img1, img2, img3 in imglist:
            imgs.append((img1, 1)) # all tag of images is 1 because we don't use tag
            imgs.append((img2, 1))
            imgs.append((img3, 1))
        self.imgs = imgs 
        self.transform = transform
        self.target_transform = target_transform
        
    def __getitem__(self, index):
        fn, label = self.imgs[index]
        # img = Image.open(fn) #no need to convert?
        img = Image.fromarray(fn).convert('RGB') 
        if self.transform is not None:
            img = self.transform(img) 
        return img, label
    def __len__(self):
        return len(self.imgs)
    

In [ ]:
def processImage(img):
    ned = cv2.imread("./E.png")
    result = cv2.matchTemplate(img, ned, cv2.TM_CCOEFF_NORMED)
    _, _, _, max_loc = cv2.minMaxLoc(result)
    test_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, test_img = cv2.threshold(test_img, 150, 255, cv2.THRESH_BINARY)

    sample1 = test_img[max_loc[1] : max_loc[1]+21, max_loc[0]-41 : max_loc[0]-20]
    sample1 = cv2.resize(sample1,(32, 32))
    sample2 = test_img[max_loc[1] : max_loc[1]+21, max_loc[0]-19 : max_loc[0]+1]
    sample2 = cv2.resize(sample2, (32, 32))
    sample3 = test_img[max_loc[1] : max_loc[1]+21, max_loc[0]+36 : max_loc[0]+57]
    sample3 = cv2.resize(sample3, (32, 32))
    return sample1, sample2, sample3

In [ ]:
# import numpy as np
# imgtest = Image.open('./frames/1/1.0-1.png')
# imgtest = Image.open('./frames/1/1.0-1.png').convert('RGB')
# plt.imshow(imgtest)
# plt.show()
# print(np.shape(imgtest))

In [ ]:
def splitSave(frame, cnt, savetodisk = False):
    test_path = './frames/1/'
    first_path = test_path + str(cnt) + '-1.png'
    second_path = test_path + str(cnt) + '-2.png'
    third_path = test_path + str(cnt) + '-3.png'
    os.makedirs(name=test_path, exist_ok=True)
    first, second, third = processImage(frame)
    first = cv2.resize(first, (32, 32))
    second = cv2.resize(second, (32, 32))
    third = cv2.resize(third, (32, 32))
    if savetodisk:
        cv2.imwrite(first_path, first)
        cv2.imwrite(second_path, second)
        cv2.imwrite(third_path, third)
    return first, second, third

def readNumber(net, sheet, show_img = False, show_result = False, test_path1 = './frames/1/', readfromdisk = False, imageslist = None):#test three images
    if readfromdisk:
        real_test = MyDataset_fromdisk(dirpath=test_path1, transform=custom_transform)
        real_testloader = torch.utils.data.DataLoader(real_test,
            batch_size = 3,
            shuffle = False)
    else:
        if imageslist is None:
            raise Exception('No imageslist found.')
        real_test = MyDataset_notfromdisk(imglist=imageslist, transform=custom_transform)
        real_testloader = torch.utils.data.DataLoader(real_test,
            batch_size = 3,
            shuffle = False)
    
    for i, batch in enumerate(real_testloader, 1):
        images = batch[0]
        if show_img:
            plt.imshow(images[0][0])
            plt.show()
            plt.imshow(images[1][0])
            plt.show()
            plt.imshow(images[2][0])
            plt.show()
        
        outputs = net(images)
        sm = nn.Softmax(dim=1)      
        sm_outputs = sm(outputs)
        probs, index = torch.max(sm_outputs, dim=1)
        first = eval(classes[index[0]])
        second = eval(classes[index[1]])
        third = eval(classes[index[2]])
        result = (first + second*0.1)*(10**third)
        sheet.write(i, 1, result)
        sheet.write(i, 0, i/10) #'result_cnt/10' means time(units of sec)
        if show_result:
            print(result, i/10) #for debug
    

In [ ]:
def clearDir(path, debug=False):# One layer delete
    if os.path.exists(path):
        for i, img in enumerate(os.listdir(path), 1):
            os.remove(path+img)
            if i%1000 == 0 and debug == True:
                print("%d files have been deleted"%(i))
        if debug == True:
            print("All files deleted.")

In [ ]:
#assist functions of `ReadExpData()`
def loadNet(netpath):
    net = Network()
    net.load_state_dict(torch.load(netpath))
    return net

def openVideo(videoname):
    video = cv2.VideoCapture("./" + videoname)
    #check if the video exists
    if not video.isOpened():
        raise Exception("Video cannot be opened.")
    return video

def showVideoInfo(videoname, video):
    #show some info of video
    print("Video: " + videoname)
    print("Total frams: " + str(video.get(7)))
    print("FPS: " + str(video.get(5)))
    #if sampling_rate = 10, it means every 0.1s we capture a photo
    sampling_rate = 10
    print("Sampling_rate: " + str(sampling_rate) + "Hz") 
    # timeF: we pick one frame every 'timeF' frames.
    # Here we pick one frame every 5 frames. 
    timeF = video.get(5) / sampling_rate 
    print("Frams needed to be extracted:" + str(video.get(7)/timeF))
    return sampling_rate, timeF, video.get(7)

def initExcelTable():
    #create and initialize an Excel table
    wb = Workbook()
    sheet1 = wb.add_sheet('Sheet 1')
    sheet1.write(0, 0, "Time/s")
    sheet1.write(0, 1, "Pressure/Pa")
    print("Time/s", "Pressure/Pa")
    return wb, sheet1

def saveResult(wb, videoname):
    if os.path.exists(videoname+"_result/"):
        clearDir(videoname+"_result/")
        os.removedirs(videoname+"_result/")    
    os.makedirs(videoname+"_result/")
    wb.save(videoname+"_result/"+'result.xls')
    
def showProgress(cur, tot):
    print("\r", end="")
    print("progress: {:.1f}%".format(cur/tot*100), end="")
    sys.stdout.flush()

In [ ]:
def readExpData(_videoname, _netpath = './result.pth', intermediate_img_folder = './frames/', savetodisk = False):
    # load the trained convolution network
    net = loadNet(_netpath)
    for videoname in _videoname:
        if os.path.exists(intermediate_img_folder):
            clearDir(intermediate_img_folder + '1/')
        video = openVideo(videoname)
        # timeF: we pick one frame every 'timeF' frames.
        # Here we pick one frame every 5 frames. 
        _, timeF, totalframes = showVideoInfo(videoname, video) 
        wb, sheet1 = initExcelTable()
        rval = True
        frame_cnt = 1
        print("Splitting the image...")
        imageslist = []
        while rval:  
            # Keep reading frames until rval=False(that is, end of file)
            rval, frame = video.read()
            if (frame_cnt % timeF == 0 and rval): 
                # take down the data
                first, second, third = splitSave(frame=frame, cnt=frame_cnt//timeF, savetodisk=savetodisk)
                if savetodisk == False:
                    imageslist.append((first, second, third))
                    # if len(imageslist) % 10 == 0:
                    #     print(len(imageslist))
                    # if len(imageslist) == 30:
                    #     break
            frame_cnt += 1
            showProgress(frame_cnt, totalframes)
            
        #save the excel table
        print("Reading the number...    ", end="")
        readNumber(net=net, sheet=sheet1, readfromdisk=savetodisk, imageslist=imageslist)
        saveResult(wb, videoname)
        print("Done!\n")

####Main function
- parameters:
  - _videoname: a list contains all names of video you want to extract experiment data. Note that all video **should be at the same directory as the program.**
  - _netpath: the name of file of the network you previously trained.

In [ ]:
#Main function: readExpData()
#for each video, you may change the variable 'videoname'
readExpData(_videoname = ["all close.MTS", "one valve_down.MTS", "one valve_leftdown.MTS", "two valves.MTS"], _netpath="./result_bsize=24, epoch=100.pth", savetodisk=False)